In [1]:
df <- read.csv("training_data.csv", stringsAsFactors=T, header = TRUE, strip.white = TRUE, sep = ",")
#df <- df[, setdiff(names(df), "X")]
#df_labels <- read.csv("training_labels.csv", header = TRUE, strip.white = TRUE, sep = ",")

In [2]:
#install.packages("dummies")
#install.packages("stringdist")
#install.packages("dplyr")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependencies ‘fansi’, ‘pkgconfig’, ‘purrr’, ‘cli’, ‘utf8’, ‘ellipsis’, ‘generics’, ‘glue’, ‘lifecycle’, ‘magrittr’, ‘R6’, ‘rlang’, ‘tibble’, ‘tidyselect’, ‘vctrs’, ‘pillar’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
library(rlang)
library(dummies)
library(stringdist)
library(dplyr)

dummies-1.5.6 provided by Decision Patterns


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
lowercase_all = function(df) {
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "factor") {
            df[,column] <<- tolower(df[,column])
        }
    }
}

In [4]:
lowercase_all(df)

In [5]:
#filter long/lat
x_up_right = 0.022
y_up_right = 40.729
x_down_left = -12.729
y_down_left = 28.138

df <- df[df$latitude > x_down_left & df$latitude < x_up_right & df$longitude > y_down_left & df$longitude < y_up_right, ] #filter pumps in atlantic fucking ocean

In [6]:
drops <- c("latitude", "longitude", "funder", "date_recorded", "gps_height", "wpt_name", "num_private", "lga", "ward", "subvillage", "region_code", "district_code", "recorded_by", "scheme_name", "waterpoint_type_group", "payment", "management", "management_group", "extraction_type_group", "extraction_type_class", "quantity_group", "quality_group", "amount_tsh", "population", "public_meeting", "scheme_management", "permit", "source_type", "source_class")
df = df[ , !(names(df) %in% drops)] #drop useless (a priori) features, maybe after feature selection remove less

In [7]:
df

,id,installer,basin,region,construction_year,extraction_type,payment_type,water_quality,quantity,source,waterpoint_type
1,69572,roman,lake nyasa,iringa,1999,gravity,annually,soft,enough,spring,communal standpipe
2,8776,grumeti,lake victoria,mara,2010,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe
3,34310,world vision,pangani,manyara,2009,gravity,per bucket,soft,enough,dam,communal standpipe multiple
4,67743,unicef,ruvuma / southern coast,mtwara,1986,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple
5,19728,artisan,lake victoria,kagera,0,gravity,never pay,soft,seasonal,rainwater harvesting,communal standpipe
6,9944,dwe,pangani,tanga,2009,submersible,per bucket,salty,enough,other,communal standpipe multiple
7,19816,dwsp,internal,shinyanga,0,swn 80,never pay,soft,enough,machine dbh,hand pump
8,54551,dwe,lake tanganyika,shinyanga,0,nira/tanira,unknown,milky,enough,shallow well,hand pump
9,53934,water aid,lake tanganyika,tabora,0,india mark ii,never pay,salty,seasonal,machine dbh,hand pump
10,46144,artisan,lake victoria,kagera,0,nira/tanira,never pay,soft,enough,shallow well,hand pump


In [8]:
work_on = setdiff(names(df), c("id"))
df[work_on][df[work_on] == 0 | df[work_on] == "-" | df[work_on] == "" | df[work_on] == "unknown"] = NA

In [9]:
sum(complete.cases(df)) #check how many rows contain at least one NA val
full_df <- na.omit(df)

na_df = df[!duplicated(rbind(df, full_df))[-seq_len(nrow(df))], ]

[1] 31383

In [10]:
na_df = dplyr::setdiff(df,full_df)
dim(na_df)

[1] 26205    11

In [11]:
similarity_test_apply = function(line) {
    NAs = is.na(line)
    idxs = which(is.na(line))
            #print(df[names(df)[idxs]])
    line2 = line[-idxs]
    most_similar = full_df[which.min(Reduce(`+`,Map(stringdist, setdiff(line2, full_df[, -idxs]), line2, method='jaccard'))),]

            #print(most_similar)
            #print("up most similar, down to change")
            #print(line)

    line[idxs] <- most_similar[idxs]
            #print(line$id)
    to_change = which(strtoi(df$id) == strtoi(line$id))
    assign('df',df,envir=.GlobalEnv)
    df[to_change, idxs] <<- most_similar[idxs]
        
}

In [35]:
#testing similarity

start.time = Sys.time()
apply(na_df[1:300,], 1, similarity_test_apply)
round(Sys.time() - start.time,5)

Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of shorter”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of shorter”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of shorter”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of shorter”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of shorter”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of shorter”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of shorter”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of shorter”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“longer argument not a multiple of length of 

$`1`
  construction_year
1              1999

$`2`
  construction_year
1              1999

$`3`
  construction_year payment_type
1              1999     annually

$`4`
  construction_year
1              1999

$`5`
  construction_year
1              1999

$`6`
  construction_year
1              1999

$`7`
  construction_year
1              1999

$`8`
  payment_type
1     annually

$`9`
  payment_type
1     annually

$`10`
  construction_year
1              1999

$`11`
  construction_year
1              1999

$`12`
  installer construction_year
1     roman              1999

$`13`
  construction_year
1              1999

$`14`
  construction_year
1              1999

$`15`
  payment_type
1     annually

$`16`
  installer payment_type water_quality
1     roman     annually          soft

$`17`
  construction_year
1              1999

$`18`
  installer construction_year
1     roman              1999

$`19`
  construction_year
1              1999

$`20`
  construction_year
1              1999

$`21`
  construction_year
1              1999

$`22`
  payment_type
1     annually

$`23`
  installer payment_type water_quality
1     roman     annually          soft

$`24`
  construction_year
1              1999

$`25`
  water_quality
1          soft

$`26`
  installer construction_year
1     roman              1999

$`27`
  installer construction_year
1     roman              1999

$`28`
  installer construction_year
1     roman              1999

$`29`
  payment_type
1     annually

$`30`
  construction_year payment_type
1              1999     annually

$`31`
  payment_type
1     annually

$`32`
  construction_year
1              1999

$`33`
  payment_type
1     annually

$`34`
  construction_year
1              1999

$`35`
  construction_year
1              1999

$`36`
  construction_year
1              1999

$`37`
  construction_year
1              1999

$`38`
  construction_year water_quality
1              1999          soft

$`39`
  construction_year
1              1999

$`40`
  payment_type
1     annually

$`41`
  construction_year
1              1999

$`42`
  construction_year
1              1999

$`43`
  installer construction_year
1     roman              1999

$`44`
  payment_type
1     annually

$`45`
  payment_type
1     annually

$`46`
  construction_year
1              1999

$`47`
  construction_year
1              1999

$`48`
  construction_year
1              1999

$`49`
  construction_year
1              1999

$`50`
  payment_type
1     annually

$`51`
  construction_year
1              1999

$`52`
  construction_year
1              1999

$`53`
  installer construction_year
1     roman              1999

$`54`
  construction_year
1              1999

$`55`
  installer
1     roman

$`56`
  construction_year payment_type
1              1999     annually

$`57`
  construction_year
1              1999

$`58`
  construction_year
1              1999

$`59`
  construction_year
1              1999

$`60`
  installer construction_year
1     roman              1999

$`61`
  construction_year payment_type
1              1999     annually

$`62`
  construction_year
1              1999

$`63`
  construction_year
1              1999

$`64`
  construction_year
1              1999

$`65`
  construction_year
1              1999

$`66`
  construction_year
1              1999

$`67`
  payment_type
1     annually

$`68`
  water_quality
1          soft

$`69`
  installer construction_year
1     roman              1999

$`70`
  construction_year
1              1999

$`71`
  payment_type
1     annually

$`72`
  installer construction_year
1     roman              1999

$`73`
  payment_type
1     annually

$`74`
  installer construction_year
1     roman              1999

$`75`
  installer
1     roman

$`76`
  payment_type
1     annually

$`77`
  construction_year
1              1999

$`78`
  construction_year payment_type
1              1999     annually

$`79`
  construction_year
1              1999

$`80`
  installer
1     roman

$`81`
  c

Time difference of 2.63173 mins

In [12]:
df[1:300,]

,id,installer,basin,region,construction_year,extraction_type,payment_type,water_quality,quantity,source,waterpoint_type
1,69572,roman,lake nyasa,iringa,1999,gravity,annually,soft,enough,spring,communal standpipe
2,8776,grumeti,lake victoria,mara,2010,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe
3,34310,world vision,pangani,manyara,2009,gravity,per bucket,soft,enough,dam,communal standpipe multiple
4,67743,unicef,ruvuma / southern coast,mtwara,1986,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple
5,19728,artisan,lake victoria,kagera,NA,gravity,never pay,soft,seasonal,rainwater harvesting,communal standpipe
6,9944,dwe,pangani,tanga,2009,submersible,per bucket,salty,enough,other,communal standpipe multiple
7,19816,dwsp,internal,shinyanga,NA,swn 80,never pay,soft,enough,machine dbh,hand pump
8,54551,dwe,lake tanganyika,shinyanga,NA,nira/tanira,NA,milky,enough,shallow well,hand pump
9,53934,water aid,lake tanganyika,tabora,NA,india mark ii,never pay,salty,seasonal,machine dbh,hand pump
10,46144,artisan,lake victoria,kagera,NA,nira/tanira,never pay,soft,enough,shallow well,hand pump


In [19]:
indx <- tail(names(sort(table(df$installer))),10)
print(indx)
to_change = which(!(full_df$installer %in% indx))
full_df[to_change, 'installer'] = 'other'
full_df

 [1] "central government" "kkkt"               "district council"  
 [4] "danida"             "commu"              "rwe"               
 [7] "hesawa"             "government"         "dwe"               
[10] "other"             


,id,installer,basin,region,construction_year,extraction_type,payment_type,water_quality,quantity,source,waterpoint_type
1,69572,other,lake nyasa,iringa,1999,gravity,annually,soft,enough,spring,communal standpipe
2,8776,other,lake victoria,mara,2010,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe
3,34310,other,pangani,manyara,2009,gravity,per bucket,soft,enough,dam,communal standpipe multiple
4,67743,other,ruvuma / southern coast,mtwara,1986,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple
6,9944,dwe,pangani,tanga,2009,submersible,per bucket,salty,enough,other,communal standpipe multiple
11,49056,other,wami / ruvu,pwani,2011,submersible,never pay,salty,enough,machine dbh,other
12,50409,danida,lake nyasa,ruvuma,1987,swn 80,on failure,soft,insufficient,shallow well,hand pump
14,50495,other,pangani,kilimanjaro,2009,gravity,monthly,soft,enough,spring,communal standpipe
16,61848,dwe,lake tanganyika,rukwa,1991,swn 80,never pay,soft,enough,machine dbh,hand pump
17,48451,dwe,rufiji,iringa,1978,gravity,monthly,soft,dry,river,communal standpipe


In [20]:
indx <- tail(names(sort(table(full_df$extraction_type))),17)
print(indx)
to_change = which(!(full_df$extraction_type %in% indx))
full_df[to_change, 'extraction_type'] = 'other'
full_df

 [1] "other - mkulima/shinyanga" "walimi"                   
 [3] "other - play pump"         "windmill"                 
 [5] "india mark iii"            "other - swn 81"           
 [7] "other - rope pump"         "afridev"                  
 [9] "ksb"                       "mono"                     
[11] "india mark ii"             "swn 80"                   
[13] "nira/tanira"               "other"                    
[15] "submersible"               "gravity"                  


,id,installer,basin,region,construction_year,extraction_type,payment_type,water_quality,quantity,source,waterpoint_type
1,69572,other,lake nyasa,iringa,1999,gravity,annually,soft,enough,spring,communal standpipe
2,8776,other,lake victoria,mara,2010,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe
3,34310,other,pangani,manyara,2009,gravity,per bucket,soft,enough,dam,communal standpipe multiple
4,67743,other,ruvuma / southern coast,mtwara,1986,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple
6,9944,dwe,pangani,tanga,2009,submersible,per bucket,salty,enough,other,communal standpipe multiple
11,49056,other,wami / ruvu,pwani,2011,submersible,never pay,salty,enough,machine dbh,other
12,50409,danida,lake nyasa,ruvuma,1987,swn 80,on failure,soft,insufficient,shallow well,hand pump
14,50495,other,pangani,kilimanjaro,2009,gravity,monthly,soft,enough,spring,communal standpipe
16,61848,dwe,lake tanganyika,rukwa,1991,swn 80,never pay,soft,enough,machine dbh,hand pump
17,48451,dwe,rufiji,iringa,1978,gravity,monthly,soft,dry,river,communal standpipe


In [22]:
dftest = full_df
dftest_ = dummy.data.frame(dftest, sep="_")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument 

In [23]:
df.pca <- prcomp(dftest_, center = TRUE,scale. = TRUE)

In [27]:
summary(df.pca)

Importance of components:
                           PC1     PC2     PC3     PC4     PC5     PC6     PC7
Standard deviation     2.31248 1.86417 1.79532 1.62256 1.61976 1.60173 1.51008
Proportion of Variance 0.06218 0.04041 0.03748 0.03061 0.03051 0.02983 0.02652
Cumulative Proportion  0.06218 0.10259 0.14007 0.17068 0.20119 0.23102 0.25754
                           PC8     PC9    PC10    PC11    PC12    PC13    PC14
Standard deviation     1.48260 1.46871 1.38214 1.36365 1.31759 1.30277 1.27072
Proportion of Variance 0.02556 0.02508 0.02221 0.02162 0.02019 0.01974 0.01878
Cumulative Proportion  0.28310 0.30818 0.33039 0.35201 0.37220 0.39194 0.41071
                          PC15   PC16    PC17    PC18   PC19    PC20    PC21
Standard deviation     1.21304 1.1948 1.17903 1.15984 1.1435 1.12950 1.11702
Proportion of Variance 0.01711 0.0166 0.01616 0.01564 0.0152 0.01483 0.01451
Cumulative Proportion  0.42782 0.4444 0.46058 0.47623 0.4914 0.50627 0.52077
                          PC22    

In [108]:
test = as.data.frame(table(df$extraction_type))
test[order(-test$Freq),]
dim(test)

,Var1,Freq
4,gravity,26696
9,nira/tanira,7361
10,other,6162
14,submersible,4688
15,swn 80,3448
8,mono,2817
5,india mark ii,2284
1,afridev,1659
7,ksb,1358
12,other - rope pump,451


[1] 17  2

In [152]:
factor_variables<-which(sapply(df[1,],class)=="character" | sapply(df[1,],class)=="logical" )
factor_variables

installer             basin            region    public_meeting 
                3                 4                 5                 7 
scheme_management            permit   extraction_type      payment_type 
                8                 9                11                12 
    water_quality          quantity       source_type      source_class 
               13                14                15                16 
  waterpoint_type 
               17

In [ ]:
other - mkulima/shinyanga

In [153]:
df_int<-df[,-factor_variables]
#summary(df)
summary(df_int)

       id          amount_tsh         population      construction_year
 Min.   :    0   Min.   :     0.0   Min.   :    0.0   Min.   :   0     
 1st Qu.:18523   1st Qu.:     0.0   1st Qu.:    0.0   1st Qu.:   0     
 Median :37055   Median :     0.0   Median :   35.0   Median :1988     
 Mean   :37106   Mean   :   327.6   Mean   :  185.6   Mean   :1342     
 3rd Qu.:55667   3rd Qu.:    30.0   3rd Qu.:  230.0   3rd Qu.:2004     
 Max.   :74247   Max.   :350000.0   Max.   :30500.0   Max.   :2013     

In [194]:
for(i in 2:ncol(df)) {
    t = as.data.frame(table(df[i]))
    print(t[order(-t$Freq),])
}
#t = as.data.frame(table(df$installer))

#t[order(-t$Freq),]

,Var1,Freq
,<fct>,<int>
378,dwe,16258
498,government,1736
568,hesawa,1356
1419,rwe,1181
257,commu,1065
307,danida,1050
354,district council,961
783,kkkt,909
202,central government,760


In [169]:
orig <- read.csv("training_data.csv", header = TRUE, strip.white = TRUE, sep = ",")

In [173]:
t = as.data.frame(table(orig$wpt_name))

tab = t[order(-t$Freq),]
tab[tab$Var1 == "none"]

,Var1
,<fct>
32437,none
35531,Shuleni
37197,Zahanati
29744,Msikitini
5143,Kanisani
1067,Bombani
35865,Sokoni
33200,Ofisini
34406,School


In [118]:
#write.csv(df, "similar_imputation.csv") #to save so we don't lose too much time

In [ ]:
### DON'T RUN CELLS FROM DOWN HERE ###

In [78]:
work_on = setdiff(names(df), c("amount_tsh", "id"))
df[work_on][df[work_on] == 0 | df[work_on] == "-" | df[work_on] == ""] = NA

In [28]:
df <-  structure(list(Age = c(23, 29, 45, 24, 28), Occupation = c("Builder", 
"Worker", "Contractor", "Engineer", "Doctor"), Nationality = c("German", 
"British", "Vietnamese", "German", "Indian")), .Names = c("Age", 
"Occupation", "Nationality"), row.names = c(NA, -5L), class = "data.frame")

newdata <- list(23,"Doctor","German")

In [29]:
df[which.min(Reduce(`+`,Map(stringdist,df, newdata,method='jaccard'))),]

,Age,Occupation,Nationality
,<dbl>,<chr>,<chr>
1,23,Builder,German


[1] 40


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
print(df[1, "latitude"])
print(df[1, "longitude"])

[1] -9.856322
[1] 34.93809


In [18]:
df

,id,amount_tsh,installer,basin,region,population,public_meeting,scheme_management,permit,construction_year,extraction_type,payment_type,water_quality,quantity,source_type,source_class,waterpoint_type
,<int>,<dbl>,<chr>,<chr>,<chr>,<int>,<lgl>,<chr>,<lgl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,6000,Roman,Lake Nyasa,Iringa,109,TRUE,VWC,FALSE,1999,gravity,annually,soft,enough,spring,groundwater,communal standpipe
2,8776,0,GRUMETI,Lake Victoria,Mara,280,NA,Other,TRUE,2010,gravity,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe
3,34310,25,World vision,Pangani,Manyara,250,TRUE,VWC,TRUE,2009,gravity,per bucket,soft,enough,dam,surface,communal standpipe multiple
4,67743,0,UNICEF,Ruvuma / Southern Coast,Mtwara,58,TRUE,VWC,TRUE,1986,submersible,never pay,soft,dry,borehole,groundwater,communal standpipe multiple
5,19728,0,Artisan,Lake Victoria,Kagera,0,TRUE,,TRUE,0,gravity,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe
6,9944,20,DWE,Pangani,Tanga,1,TRUE,VWC,TRUE,2009,submersible,per bucket,salty,enough,other,unknown,communal standpipe multiple
7,19816,0,DWSP,Internal,Shinyanga,0,TRUE,VWC,TRUE,0,swn 80,never pay,soft,enough,borehole,groundwater,hand pump
8,54551,0,DWE,Lake Tanganyika,Shinyanga,0,TRUE,,TRUE,0,nira/tanira,unknown,milky,enough,shallow well,groundwater,hand pump
9,53934,0,Water Aid,Lake Tanganyika,Tabora,0,TRUE,VWC,TRUE,0,india mark ii,never pay,salty,seasonal,borehole,groundwater,hand pump


In [ ]:
categories <- unique(df$basin) 

In [ ]:
region, lga, ward, subvillage, basin, wpt_name, 

In [ ]:
upper left -0.6721467040207111, 28.299198544014903
upper right 0.022237, 40.729917
lower right -12.609672, 41.940606
lower left -12.729036 = x = latitude, 28.138472 = y = longitude


if (x > x1 and x < x2 and y > y1 and y < y2)

In [12]:
df$funder = tolower(df$funder)
df

,id,amount_tsh,funder,installer,basin,region,population,public_meeting,scheme_management,permit,construction_year,extraction_type,payment_type,water_quality,quantity,source_type,source_class,waterpoint_type
,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<lgl>,<chr>,<lgl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,6000,roman,Roman,Lake Nyasa,Iringa,109,TRUE,VWC,FALSE,1999,gravity,annually,soft,enough,spring,groundwater,communal standpipe
2,8776,0,grumeti,GRUMETI,Lake Victoria,Mara,280,NA,Other,TRUE,2010,gravity,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe
3,34310,25,lottery club,World vision,Pangani,Manyara,250,TRUE,VWC,TRUE,2009,gravity,per bucket,soft,enough,dam,surface,communal standpipe multiple
4,67743,0,unicef,UNICEF,Ruvuma / Southern Coast,Mtwara,58,TRUE,VWC,TRUE,1986,submersible,never pay,soft,dry,borehole,groundwater,communal standpipe multiple
5,19728,0,action in a,Artisan,Lake Victoria,Kagera,0,TRUE,,TRUE,0,gravity,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe
6,9944,20,mkinga distric coun,DWE,Pangani,Tanga,1,TRUE,VWC,TRUE,2009,submersible,per bucket,salty,enough,other,unknown,communal standpipe multiple
7,19816,0,dwsp,DWSP,Internal,Shinyanga,0,TRUE,VWC,TRUE,0,swn 80,never pay,soft,enough,borehole,groundwater,hand pump
8,54551,0,rwssp,DWE,Lake Tanganyika,Shinyanga,0,TRUE,,TRUE,0,nira/tanira,unknown,milky,enough,shallow well,groundwater,hand pump
9,53934,0,wateraid,Water Aid,Lake Tanganyika,Tabora,0,TRUE,VWC,TRUE,0,india mark ii,never pay,salty,seasonal,borehole,groundwater,hand pump


In [ ]:
missing_imputation_numbers = function(df) {
    #print(df)
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "integer" | columnClass == "numeric") {
            print(columnClass)
            df[is.na(df[,column]), column] <<- mean(df[,column], na.rm = TRUE)
        }
     }
}

In [4]:
missing_imputation_logical = function(df) {
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "logical") {
            categories <- unique(df[,column]) 
            df[is.na(df[,column]), column] <<- sample(categories, 1)
        }
    }
}

In [ ]:
missing_imputation_numbers = function(df) {
    #print(df)
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "integer" | columnClass == "numeric") {
            print(columnClass)
            df[is.na(df[,column]), column] <<- mean(df[,column], na.rm = TRUE)
        }
     }
}

In [44]:
#one-hot enc

dftest = df
keep = c("basin", "region")
print(names(dftest))
dftest_ = dummy.data.frame(dftest[,keep], sep="_")

 [1] "id"                "amount_tsh"        "installer"        
 [4] "basin"             "region"            "population"       
 [7] "public_meeting"    "scheme_management" "permit"           
[10] "construction_year" "extraction_type"   "payment_type"     
[13] "water_quality"     "quantity"          "source_type"      
[16] "source_class"      "waterpoint_type"  


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
